In [2]:
# 연습문제 4_1. 네이버 블로그 검색 건수 입력받아 페이지 변경하면서 요약 데이터 수집 후 다양한 형식의 파일로 저장하기

print("=" *100)
print(" 연습문제 13-2: 네이버 블로그 크롤러 : 상세검색옵션 사용하여 페이지를 변경하면서 크롤링 후 요약 내용 저장하기")
print("=" *100)

#Step 0. 필요한 모듈과 라이브러리를 로딩하고 검색어를 입력 받습니다

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys        # system 설정을 변경하기 위해 필요합니다

# 엑셀 형식과 csv 형식으로 저장하기 위해 표 형태로 데이터를 만들어야 합니다.
# 그렇게 하기 위해 numpy , pandas 패키지를 설치해야 합니다.
import numpy           # cmd 창에서 pip install numpy==1.15.1 로 설치 먼저 하세요
import pandas  as pd    # cmd 창에서 pip install pandas 로 설치 먼저 하세요

# 페이지 번호 계산을 위해 사용합니다
import math    

#필요한 정보를 입력 받습니다
query_txt = input('1.크롤링할 키워드는 무엇입니까?: ')
query_txt2 = input('2.포함할 키워드를 입력하세요: ')
query_txt3 = input('3.제외할 키워드를 입력하세요: ')

cnt = int(input('4.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 페이지 수 


start_date = input('5.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :')
end_date = input('6.조회를 종료할 날짜를 입력하세요(예:2017-12-31): ')

f_name = input('7.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')
fc_name = input('8.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')
fx_name = input('9.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: ')


#Step 1. 크롬 드라이버를 사용해서 웹 브라우저를 실행합니다.
path = "c:/temp/chromedriver_240/chromedriver.exe"
driver = webdriver.Chrome(path)

driver.get('http://www.naver.com')
time.sleep(2)

s_time = time.time( )     # 검색을 시작하는 시점의 timestamp 를 지정합니다

#Step 2. 네이버 검색창에 입력 받은 검색어를 넣고 검색을 실행합니다
element = driver.find_element_by_id("query")
element.send_keys(query_txt)
element.submit()

#Step 3. 아래의 블로그 링크를 선택합니다
driver.find_element_by_link_text("블로그").click( )

#Step 4. 오른쪽의 검색 옵션 버튼 클릭
driver.find_element_by_id("_search_option_btn").click( )


#Step 5. 정렬 버튼 클릭하기
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/a""").click( )  # 정렬 버튼의 xpath 클릭
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[1]/div/ul/li[2]/a""").click( ) # 최신순 xpath

#Step 6. 기간 버튼 클릭하여 날짜 입력하기
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[2]/a""").click()
time.sleep(2)


# 이 부분이 아주 중요합니다.
# 날짜 부분에 날짜를 입력할 때 입력 속도가 너무 빠를 경우 날짜가 입력이 되다가 
# 오타가 나오는 경우가 많습니다.
# 그래서 날짜를 입력할 때 for 반복문을 사용해서 1 글자씩 입력하도록 코딩했습니다.

# 직접 입력 시작 날짜 클릭
s_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""")     
driver.find_element_by_xpath("""//*[@id="blog_input_period_begin"]""").click()
s_date.clear( )  # 날짜 입력 부분에 기존에 입력되어 있던 날짜를 제거합니다. 
time.sleep(1)

# 아래 코드가 날짜를 for 반복문으로 1 글자씩 입력하는 부분입니다.
for c in start_date:
    s_date.send_keys(c)
    time.sleep(0.3)

e_date = driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""")
driver.find_element_by_xpath("""//*[@id="blog_input_period_end"]""").click()
e_date.clear()
time.sleep(1)

for c in end_date:
    e_date.send_keys(c)
    time.sleep(0.3)

# Step7. 날짜 입력을 완료한 후 적용하기 버튼을 클릭 합니다.  
driver.find_element_by_xpath("""//*[@id="_nx_option_date"]/div/span/button/span""").click()
time.sleep(2)

# Step 8. 상세 검색버튼을 클릭 후 포함할 단어와 제외할 단어들을 설정합니다.
driver.find_element_by_xpath("""//*[@id="snb"]/div/ul/li[7]/a""").click( )
ele2 = driver.find_element_by_id("inpop2")
ele2.send_keys(query_txt2)

ele3 = driver.find_element_by_id("inpop3")
ele3.send_keys(query_txt3)

driver.find_element_by_xpath("""//*[@id="_nx_option_detail"]/div/div[3]/button[1]""").click( )

# Step 9. 현재 페이지의 내용을 저장 목록을 만든 후 목록에 있는 내용을 파일에 저장하기

no2=[]           # 게시글 번호 컬럼
title2=[ ]       # 게시글 제목 컬럼
contents2=[ ]    # 게시글 내용 컬럼
intro2=[ ]       # 블로그 소개 컬럼
addr2=[ ]        # 블로그 주소 컬럼

no = 1           # 게시글 번호 초기값

for x in range(1,page_cnt+1) :
    print("\n")
    print("%s 페이지 내용 수집 시작합니다 =======================" %x)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    blog_list = soup.select('ul > li > dl ')

    for i in blog_list:
        
        f = open(f_name, 'a',encoding='UTF-8')

        # 1.게시글 번호
        print("\n")
        no2.append(no)
        f.write("1.번호 :" + str(no) + "\n")
        print("1.번호 : %s" %no)

        # 2. 게시글 제목
        title=i.find('a').get_text()
        title2.append(title)
        f.write("2.제목 :" + str(title) + "\n")
        print("2.제목 : %s" %title)
        
        # 3. 게시글 내용
        contents = i.find('dd','sh_blog_passage').get_text().replace("\n","")                                  
        contents2.append(contents) 
        f.write("3.내용 :" + str(contents) + "\n")
        print("3.내용 : %s" %contents)
        
        # 4. 블로그 소개
        intro = i.find('span','inline').find('a').get_text()
        intro2.append(intro)
        f.write("4.소개 :" + str(intro) + "\n")
        print("4.소개 : %s" %intro)
        
        # 5. 블로그 주소
        addr = i.find('span','inline').find('a')['href']
        addr2.append(addr)
        f.write("5.블로그 주소 :" + str(addr) + "\n")
        f.write("\n")
        print("5.블로그 주소 : %s" %addr)

        f.close( )

        no += 1
        
        if no > cnt :
            break
        
    time.sleep(2)        # 페이지 변경 전 2초 대기 

    x += 1

    if x > page_cnt :
        break
    
    driver.find_element_by_link_text("""%s""" %x).click() # 다음 페이지번호 클릭

# Step 10. 출력 결과를 다양한 형식으로 저장하기

naver_blog = pd.DataFrame()
naver_blog['번호']=no2
naver_blog['제목']=title2
naver_blog['내용']=contents2
naver_blog['블로그소개']=intro2
naver_blog['블로그주소']=addr2

# csv 형태로 저장하기
naver_blog.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
naver_blog.to_excel(fx_name,index=False)

e_time = time.time( )     # 검색이 종료된 시점의 timestamp 를 지정합니다
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no-1))
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("파일 저장 완료: txt 파일명 : %s " %f_name)
print("파일 저장 완료: csv 파일명 : %s " %fc_name)
print("파일 저장 완료: xls 파일명 : %s " %fx_name)
print("=" *80)

driver.close( )


 연습문제 13-2: 네이버 블로그 크롤러 : 상세검색옵션 사용하여 페이지를 변경하면서 크롤링 후 요약 내용 저장하기
1.크롤링할 키워드는 무엇입니까?: 여름여행
2.포함할 키워드를 입력하세요: 국내
3.제외할 키워드를 입력하세요: 해외
4.크롤링 할 건수는 몇건입니까?: 50
5.조회를 시작할 날짜를 입력하세요(예:2017-01-01) :2019-01-01
6.조회를 종료할 날짜를 입력하세요(예:2017-12-31): 2019-12-31
7.txt 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\temp\aaa.txt
8.csv 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\temp\aaa.csv
9.xls 형태로 저장할 경로와 파일명을 확장자 포함해서 쓰세요: c:\temp\aaa.xls


1 페이지 내용 수집 시작합니다 =======================


1.번호 : 1
2.제목 : [선공개] 2019 사계절의 베스트 순간
3.내용 : 국내에서 가장 좋아하는 여행지인 봄의 여행지와 엄마랑 자주 함께 해서 기억에 남는 여름의 곳곳, 그리운 이를 보러 가는 가을의 풍경, 아지트 바다로 자주 찾을 것...
4.소개 : 에디터 휘서의 은은...
5.블로그 주소 : https://blog.naver.com/ohwiseo


1.번호 : 2
2.제목 : 마시안제빵소 앞 마시안해변 올해의 마지막 낙조
3.내용 : [국내 여행] 마시안제빵소 앞 마시안해변 올해의 마지막 낙조 글 | 사진 세상탐닉... 역주행을 하더라도 최근 글부터 올리고 있는데, 뜬금없이 여름 여행기가 올라와도...
4.소개 : 내 삶의 향기가 가득하기를...
5.블로그 주소 : https://shiyou1111.blog.me


1.번호 : 3
2.제목 : 스텔스 출연진 줄거리 트리비아
3.내용 : 미국내에서 제기됐다. 미국 과학 잡지 스텔스 영화에는 스텔스기가 없다? 찬 수퍼울트라 고공하이테크액션 영화 스텔스 이 무더운 여름..여행도 못간...
4.소개 : 응



5 페이지 내용 수집 시작합니다 =======================


1.번호 : 41
2.제목 : 애정한다오 bag) 가방덕후 인증 포스팅
3.내용 : 지난 여름, 여행가며 엘깡백, 코지코티지, 르백 들고 가서 가방덕후질 해보던... ㅋㅋ #린보공방 #린보가방 #linbo 요 사코슈백도 제가 가진 컬러는 국내에서도 두개뿐...
4.소개 : 농부아내☆별이
5.블로그 주소 : https://blog.naver.com/lolog3


1.번호 : 42
2.제목 : 대전 장태산 자연휴양림 (산책로 형제산 등산코스 등산지도...
3.내용 : 분들이 여행코스로 찾으시고 계신데요. 여행상품으로 단체로 오시는 분들이 무척... 한 여름에 와도 시원한 그늘과 함께 나무의 향기가 좋아 여름 여행지 명소이기도...
4.소개 : 추억이 될 일상을 남기고 있...
5.블로그 주소 : https://blog.naver.com/459745


1.번호 : 43
2.제목 : [1박2일 양양] 힙한 여행지 추천, 양양 서피비치...
3.내용 : 1박2일 양양 여름 여행 7월 16일 ~ 7월 17일 1박 2일 단양여행을 마치고 다시 길을... 국내여행도 좋아좋아 ღ’ᴗ’ღ 제가 다녀온 곳들 전체적인 위치 보시라고 지도...
4.소개 : Mylog
5.블로그 주소 : https://blog.naver.com/hwhw107


1.번호 : 44
2.제목 : 12월 국내여행지 추천 BSET3
3.내용 : 특히 경주는 봄, 여름. 가을. 겨울 할것없이 우리나라 국민들에게 사랑을 받는... 떠나보면 여름 여행과는 또 다른 즐거움을 선사하는게 12월 국내여행지라고...
4.소개 : 연예다반사
5.블로그 주소 : http://acoustic42.tistory.com/


1.번호 : 45
2.제목 : [국내여행한달] 내 고향 원주, 그리고 템플스테이
3.내용 : 원주 짱 조용하고 한적하고~ 그 여름밤의 느낌이 그립다 흑흑.. 원주 둘째 날 - 역사... 구룡사템플스테이 #원주 #원주여행 #